In [ ]:
# Download Dataset Katarak
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download jr2ngb/cataractdataset
! chmod 600 kaggle.json
! ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle
! kaggle datasets download jr2ngb/cataractdataset

100% 3.34G/3.34G [00:41<00:00, 56.8MB/s]
100% 3.34G/3.34G [00:41<00:00, 85.6MB/s]
cataractdataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Proses Ekstrak Dataset
import zipfile
import os

with zipfile.ZipFile('cataractdataset.zip','r') as z:
  z.extractall('./')

os.listdir()